## **MASTERGIS: Exploración de metadatos II**
## **ee.Geometry, ee.Feature y ee.FeatureCollection**

En esta lectura, aprenderemos sobre:

- Como obtener metadatos en ee.Geometry.
- Como obtener metadatos en ee.Feature.
- Como obtener metadatos en ee.FeatureCollection.

### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"1//04QradBm7IZA7CgYIARAAGAQSNwF-L9IrIJqgQs7wi3aA6F8eduoZS2sxG62FOlQhV1s4lW1y_7mexnYKk-fmjG_ZYNoCAyh2RDw"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **3) Metadatos ee.Geometry y ee.Feature**

Para ver la información sobre una geometría, solo utilize **.getInfo**. Para acceder a la información de forma programada. Earth Engine proporciona varios métodos:




In [0]:
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

In [18]:
print('Polygon printout: ', polygon.getInfo())

# Imprime el area del poligono en kilometros cuadrados.
print('Polygon area: ', polygon.area().divide(1000 * 1000).getInfo())

# Imprimir el perímetro del polígono en kilometros.
print('Polygon perimeter: ', polygon.perimeter().divide(1000).getInfo())

# Imprime la geometria como una cadena GeoJSON.
print('Polygon GeoJSON: ', polygon.toGeoJSONString())

# Imprima el 'tipo' GeoJSON
print('Geometry type: ', polygon.type().getInfo())

# Imprimir las coordenadas como listas
print('Polygon coordinates: ', polygon.coordinates().getInfo())

Polygon GeoJSON:  {"type": "Polygon", "coordinates": [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]], "evenOdd": true}


In [21]:
ee_demop = ee.Feature(polygon)
ee_demop = ee_demop.set('area2',10000)
ee_demop.get('area2').getInfo()

10000

### **4) Metadatos ee.FeatureCollection**


In [27]:
from pprint import pprint

# Cargar los datos de cuencas
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06')

# Verifique el numero de cuencas hidrograficas despues de filtrar por ubicacion.
print('Count after filter:', sheds.size().getInfo())

print('Primer feature: \n')
pprint(sheds.first().toDictionary().getInfo())

Count after filter: 387
Primer feature: 

{'areaacres': '17935746.85',
 'areasqkm': '72583.46',
 'gnis_id': '',
 'huc6': '190301',
 'loaddate': '20120611075458',
 'metasource': '',
 'name': 'Aleutian Islands',
 'shape_area': '9.9308571861444',
 'shape_leng': '81.5083456556245',
 'sourcedata': '',
 'sourcefeat': '',
 'sourceorig': '',
 'states': 'AK',
 'tnmid': '{7D8E9EC1-6A0D-4FED-8063-94688560AC75}'}


In [0]:
ee.Feature(sheds.toList(2).get(1)).toDictionary().getInfo()

In [40]:
ee.Feature(sheds.toList(40).get(39)).toDictionary().getInfo()


{'areaacres': '2836020.87',
 'areasqkm': '11476.98',
 'gnis_id': '',
 'huc6': '080403',
 'loaddate': '20120611075458',
 'metasource': '',
 'name': 'Lower Red',
 'shape_area': '1.09247927172349',
 'shape_leng': '8.08110991380453',
 'sourcedata': '',
 'sourcefeat': '',
 'sourceorig': '',
 'states': 'LA',
 'tnmid': '{2317909B-88CD-4656-9F58-E8CAC7F17614}'}